In [14]:
import face_recognition
import numpy as np
import cv2, queue, threading, time
import requests, os, re
from datetime import datetime



class VideoCapture:
    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.q = queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

    
    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()   
                except queue.Empty:
                    pass
            self.q.put(frame)

    def read(self):
        return self.q.get()

video_capture = VideoCapture(0)


known_face_encodings = []
known_face_names = []
known_faces_filenames = []

for (dirpath, dirnames, filenames) in os.walk('ImagesAttendance/'):
    known_faces_filenames.extend(filenames)
    break

for filename in known_faces_filenames:
    face = face_recognition.load_image_file('ImagesAttendance/' + filename)
    known_face_names.append(re.sub("[0-9]",'', filename[:-4]))
    known_face_encodings.append(face_recognition.face_encodings(face)[0])



face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

face_names = []

while True:
   
    frame = video_capture.read()
    
    
    if process_this_frame:
        
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        
        
        

        
        
        for face_encoding in face_encodings:
            
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            if name not in face_names:
                face_names.append(name)
        for (top, right, bottom, left), name in zip(face_locations, face_names):

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)


            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX

            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            arrival_time = time.strftime('%H:%M:%S', time.localtime())
            departure_time = time.strftime('%H:%M:%S', time.localtime())
            FMT='%H:%M:%S'
            date= time.strftime('%Y-%m-%d', time.localtime())


            """"import datetime
            if duration==(str(datetime.timedelta(seconds=28800))) :
                print("person has not gone early")
            elif duration==(str(datetime.timedelta(seconds=0))):
                print("person is absent")
            else:
                print("person is late")"""

    
    cv2.imshow('Video', frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        


cv2.destroyAllWindows()
print(face_names)
with open('Attendance.csv','a+') as f:
    for name in face_names:
        duration=datetime.strptime(departure_time,FMT)-datetime.strptime(arrival_time,FMT)
        f.writelines(f'\n{name},{date},{arrival_time},{departure_time},{duration}')
        
    
        
        

        
        

['Unknown', 'kartik-aaryan']


In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)